# Instance management (of simulation objects) and shutdown()

The following is taken from the end of the finmag/src/finmag/sim/sim.py test:

```
"""28 March 2015: running jenkins tests has started to fail some weeks
ago wiht an error message about too many open files. We find that the
simulation specific log-handler files are not closed. If we run many
simulations in the same python session, the number of open files
increases. It is not clear whether this is the problem, but that is the current working assumption.

Part of the problem is that self.close_logfile() is never called, and
thus the file is not closed.

As part of the investigation, we found that the simulation objects
also are not garbage collected. [Marijan reported memory problems when
he has many simulation objects created in a for-loop.] This is due to
other references to the simulation object existing when it goes out of
scope. For example, the Table writer functions have a reference to the
simulation object. We thus have cyclic references, and therefore the
garbage collection never removes the simulation objects, and thus
__del__(self) is never called.

As long as we stick to passing a reference to the simulation object
around in our code, we can not close the logfile automatically in the
__del__ method, because the delete method never executes due to the
cyclic references. We want to stick to using references to the
simulation object as this is very convenient and allows writing the
most generic code (i.e. use user-defined functions that get the
simulation object passed as an argument).

Sticking to this approach, we need an additional function to 'delete'
the simulation object. What this actually needs to do is to remove the
cyclic references so that the object can be garbage collected when it
goes out of scope. (The same function might as well close the logfile
handler, thus addressing the original problem.)

This commit introduced a method Simulation.shutdown() which provides
this functionality and (at least for simple examples) removes cyclic
references from the simulation object.

It also provides other goodies that will hopefully allow us to monitor
the existence of 'un-garbage-collected' simulation objects, and
convenience functions to shut them down, for example as part of our
test suite. This is hopeful thinking at the moment and needs more
testing to see that it delivers.
"""
```

Here is an example :

In [1]:
import finmag

[2015-03-29 20:47:40] INFO: Finmag logging output will be appended to file: '/home/fangohr/.finmag/global.log'
[2015-03-29 20:47:40] DEBUG: Building modules in 'native'...
[2015-03-29 20:47:40] DEBUG: FinMag          5851:afed2cd505ac94370b511a3398226eeb37aebaf0
[2015-03-29 20:47:40] DEBUG: Dolfin          1.4.0                Matplotlib      lazily loaded       
[2015-03-29 20:47:40] DEBUG: Numpy           1.8.2                Scipy           0.15.1              
[2015-03-29 20:47:40] DEBUG: IPython         2.4.0                Python          2.7.6               
[2015-03-29 20:47:40] DEBUG: Paraview        4.0.1-1ubuntu1       Sundials        2.5.0               
[2015-03-29 20:47:40] DEBUG: Boost-Python    <unknown>            Linux           Ubuntu 14.04.1 LTS  
[2015-03-29 20:47:40] DEBUG: Registering debug signal handler. Press Ctrl-Z any time to stop execution and jump into the debugger.


In [2]:
s1 = finmag.example.barmini(name='s1')

[2015-03-29 20:47:40] INFO: Finmag logging output will be written to file: '/home/fangohr/hg/finmag/doc/ipython_notebooks_src/s1.log' (any old content will be overwritten).
[2015-03-29 20:47:40] DEBUG: Creating DataWriter for file 's1.ndt'
[2015-03-29 20:47:40] INFO: Creating Sim object name='s1', instance_id=0 (rank=0/1).
[2015-03-29 20:47:40] DEBUG:    Total number of Sim objects in this session: 1
[2015-03-29 20:47:40] INFO: <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:40] DEBUG: Creating LLG object.
[2015-03-29 20:47:40] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-03-29 20:47:40] DEBUG: Adding interaction Exchange to simulation.
[2015-03-29 20:47:40] DEBUG: Creating Demag object with solver 'FK'.
[2015-03-29 20:47:40] DEBUG: Demag parameters now: {'phi_1': {'absolute_tolerance': 1e-06, 'relative_tolerance': 1e-06, 'maximum_iterations': 10000}, 'phi_2': {'absolute_tolerance': 1e-06, 'rela

Attempting to del the object s1 (pretending it goes out of scope), will not result in its garbage collection:

In [3]:
del s1

## We need to remove cyclic references using the shutdown method():

In [4]:
s2 = finmag.example.barmini(name='s2')
s2.shutdown()
del s2

[2015-03-29 20:47:40] INFO: Finmag logging output will be written to file: '/home/fangohr/hg/finmag/doc/ipython_notebooks_src/s2.log' (any old content will be overwritten).
[2015-03-29 20:47:40] DEBUG: Creating DataWriter for file 's2.ndt'
[2015-03-29 20:47:40] INFO: Creating Sim object name='s2', instance_id=1 (rank=0/1).
[2015-03-29 20:47:40] DEBUG:    Total number of Sim objects in this session: 2
[2015-03-29 20:47:40] INFO: <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:40] DEBUG: Creating LLG object.
[2015-03-29 20:47:40] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-03-29 20:47:40] DEBUG: Adding interaction Exchange to simulation.
[2015-03-29 20:47:40] DEBUG: Creating Demag object with solver 'FK'.
[2015-03-29 20:47:40] DEBUG: Demag parameters now: {'phi_1': {'absolute_tolerance': 1e-06, 'relative_tolerance': 1e-06, 'maximum_iterations': 10000}, 'phi_2': {'absolute_tolerance': 1e-06, 'rela

__del__(): Simulation object name='s2' (instance_id=1) is going out of scope


## Other convenience tools

There is also a convenience 'class' method to show and delete (as much as possible) other simulations

In [5]:
for i in range(3):
    finmag.example.barmini(name='%d' % i)
    
s4 = finmag.example.barmini(name='s4')

[2015-03-29 20:47:41] INFO: Finmag logging output will be written to file: '/home/fangohr/hg/finmag/doc/ipython_notebooks_src/0.log' (any old content will be overwritten).
[2015-03-29 20:47:41] DEBUG: Creating DataWriter for file '0.ndt'
[2015-03-29 20:47:41] INFO: Creating Sim object name='0', instance_id=2 (rank=0/1).
[2015-03-29 20:47:41] DEBUG:    Total number of Sim objects in this session: 2
[2015-03-29 20:47:41] INFO: <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:41] DEBUG: Creating LLG object.
[2015-03-29 20:47:41] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-03-29 20:47:41] DEBUG: Adding interaction Exchange to simulation.
[2015-03-29 20:47:41] DEBUG: Creating Demag object with solver 'FK'.
[2015-03-29 20:47:41] DEBUG: Demag parameters now: {'phi_1': {'absolute_tolerance': 1e-06, 'relative_tolerance': 1e-06, 'maximum_iterations': 10000}, 'phi_2': {'absolute_tolerance': 1e-06, 'relativ

## Class method to show all active simulation objects:

In [6]:
finmag.Simulation.instances_list_all()

[2015-03-29 20:47:41] INFO: Showing all Simulation object instances:
[2015-03-29 20:47:41] INFO:     sim instance_id=0: name='s1'
[2015-03-29 20:47:41] INFO:     sim instance_id=2: name='0'
[2015-03-29 20:47:41] INFO:     sim instance_id=3: name='1'
[2015-03-29 20:47:41] INFO:     sim instance_id=4: name='2'
[2015-03-29 20:47:41] INFO:     sim instance_id=5: name='s4'


Class method to (attempt) to delete all active simulation objects:

In [7]:
finmag.Simulation.instances_delete_all()

[2015-03-29 20:47:41] INFO: instances_delete_all() starting:
[2015-03-29 20:47:41] INFO: Shutting down Simulation object finmag.Simulation(name='s1', instance_id=0) with <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:41] DEBUG: 4 other Simulation instances alive.
[2015-03-29 20:47:41] DEBUG:    shutdown(): 1-refcount 6 for s1
[2015-03-29 20:47:41] DEBUG: 'Deletinging all get methods in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for H_Demag in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for dmdt in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for E_total in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for H_Exchange in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for m in TableWriter(name=s1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for steps in TableWriter(name

__del__(): Simulation object name='s1' (instance_id=0) is going out of scope
__del__(): Simulation object name='0' (instance_id=2) is going out of scope

[2015-03-29 20:47:41] INFO: Shutting down Simulation object finmag.Simulation(name='1', instance_id=3) with <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:41] DEBUG: 2 other Simulation instances alive.
[2015-03-29 20:47:41] DEBUG:    shutdown(): 1-refcount 6 for 1
[2015-03-29 20:47:41] DEBUG: 'Deletinging all get methods in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for H_Demag in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for dmdt in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for E_total in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for H_Exchange in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for m in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for steps in TableWriter(name=1.ndt)
[2015-03-29 20:47:41] DEBUG: 'Deleting' get method for E_Exch


__del__(): Simulation object name='1' (instance_id=3) is going out of scope

[2015-03-29 20:47:41] INFO: Shutting down Simulation object finmag.Simulation(name='2', instance_id=4) with <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:42] DEBUG: 1 other Simulation instances alive.
[2015-03-29 20:47:42] DEBUG:    shutdown(): 1-refcount 6 for 2
[2015-03-29 20:47:42] DEBUG: 'Deletinging all get methods in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for H_Demag in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for dmdt in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for E_total in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for H_Exchange in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for m in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for steps in TableWriter(name=2.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for E_Exch


__del__(): Simulation object name='2' (instance_id=4) is going out of scope

[2015-03-29 20:47:42] INFO: Shutting down Simulation object finmag.Simulation(name='s4', instance_id=5) with <Mesh of topological dimension 3 (tetrahedra) with 45 vertices and 96 cells, ordered>
[2015-03-29 20:47:42] DEBUG: 0 other Simulation instances alive.
[2015-03-29 20:47:42] DEBUG:    shutdown(): 1-refcount 7 for s4
[2015-03-29 20:47:42] DEBUG: 'Deletinging all get methods in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for H_Demag in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for dmdt in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for E_total in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for H_Exchange in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for m in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method for steps in TableWriter(name=s4.ndt)
[2015-03-29 20:47:42] DEBUG: 'Deleting' get method f

s4 cannot be garbage collected, because the variable s4 is a reference to it. Once we delete it, the object should be garbage collected:

In [8]:
del s4

__del__(): Simulation object name='s4' (instance_id=5) is going out of scope


## Summary

If we use multiple simulation objects, it is recommended to use the ``shutdown()`` method when the object is not needed anymore. (See example below). This is likely to be only important if simulations are created in a for loop in a single session, but good practice to shutdown an object when it is not needed anymore. 

Typical example:


In [9]:
import finmag

In [10]:
# create some simulation
s = finmag.example.bar()

[2015-03-29 20:47:42] INFO: Finmag logging output will be written to file: '/home/fangohr/hg/finmag/doc/ipython_notebooks_src/bar.log' (any old content will be overwritten).
[2015-03-29 20:47:42] DEBUG: Creating DataWriter for file 'bar.ndt'
[2015-03-29 20:47:42] INFO: Creating Sim object name='bar', instance_id=6 (rank=0/1).
[2015-03-29 20:47:42] DEBUG:    Total number of Sim objects in this session: 1
[2015-03-29 20:47:42] INFO: <Mesh of topological dimension 3 (tetrahedra) with 13056 vertices and 67500 cells, ordered>
[2015-03-29 20:47:42] DEBUG: Creating LLG object.
[2015-03-29 20:47:43] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-03-29 20:47:43] DEBUG: Adding interaction Exchange to simulation.
[2015-03-29 20:47:44] DEBUG: Creating Demag object with solver 'FK'.
[2015-03-29 20:47:44] DEBUG: Demag parameters now: {'phi_1': {'absolute_tolerance': 1e-06, 'relative_tolerance': 1e-06, 'maximum_iterations': 10000}, 'phi_2': {'absolute_tolerance': 1e-

In [11]:
# do some work with the simulation
s.run_until(1e-15)
# when all is finished, shut it down

[2015-03-29 20:47:55] INFO: Simulation will run until t = 1e-15 s.
[2015-03-29 20:47:55] INFO: Creating integrator with backend sundials and arguments {'reltol': 1e-06, 'abstol': 1e-06}.
[2015-03-29 20:47:55] DEBUG: Updating get method for steps in TableWriter(name=bar.ndt)
[2015-03-29 20:47:55] DEBUG: Updating get method for last_step_dt in TableWriter(name=bar.ndt)
[2015-03-29 20:47:55] DEBUG: Updating get method for dmdt in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] INFO: Simulation has reached time t = 1e-15 s.


In [12]:
# save data, and prepare to destruct simulation object:
s.shutdown()

[2015-03-29 20:47:56] INFO: Shutting down Simulation object finmag.Simulation(name='bar', instance_id=6) with <Mesh of topological dimension 3 (tetrahedra) with 13056 vertices and 67500 cells, ordered>
[2015-03-29 20:47:56] DEBUG: 0 other Simulation instances alive.
[2015-03-29 20:47:56] DEBUG:    shutdown(): 1-refcount 6 for bar
[2015-03-29 20:47:56] DEBUG: 'Deletinging all get methods in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for H_Demag in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for dmdt in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for E_total in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for H_Exchange in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for m in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deleting' get method for steps in TableWriter(name=bar.ndt)
[2015-03-29 20:47:56] DEBUG: 'Deletin

4

if desired, delete (last) reference to simulation object, or wait for it to go out of scope: 

In [13]:
del s

__del__(): Simulation object name='bar' (instance_id=6) is going out of scope
